In [2]:
import pyaerocom as pya
from pyaerocom.io.read_emep import ReadEMEP
import glob
import os
import xarray as xr

In [5]:
data_id = 'EMEPrv4.33-met2010'

## Lustre
# base = '/lustre/storeB/project/fou/kl/emep/ModelRuns/2020_AerocomHIST' # Directory containing files to be converted
# # Not sure where to store temporary files at lustre # out_dir = '' # Directory for storing converted files

## Locally
base = '/home/eirikg/Desktop/pyaerocom/data/2020_AerocomHIST/' # Directory containing files to be converted
out_dir = os.path.join(base, 'processed') # Directory for storing converted files

if not os.path.exists(out_dir):
   os.mkdir(out_dir)

In [8]:
# Get list of all directories in base
files = os.listdir(base)
dirs = []
for file in files:
    if os.path.isdir(base + file):
        if len(file.split('_')) == 3:
            dirs.append(file)

In [11]:
## Available years
years = []
for directory in dirs:
    years.append(int(directory.split('_')[0]))

In [13]:
# Select first folder and its monthly resolution file, and get year from folder name
file = os.path.join(base + dirs[0], 'Base_month.nc')
year = int(dirs[0].split('_')[0])

In [25]:
# Create reader. var_file contains the mapping from EMEP -> Aerocom
reader = ReadEMEP(file, data_id=data_id, var_file=os.path.join(base, 'vars_sorted.sh'))

In [9]:
# List all unique EMEP variable prefixes
aero_emep = reader.map_aero_emep()
types = []
for key, val in aero_emep.items():
    type_emep = (val.split('_')[0])
    if type_emep not in types:
        types.append(type_emep)

In [24]:
# Map EMEP variable prefix to vertical type
map_prefix_aero = {'AOD': 'Column' ,'AAOD' : 'Column', 'Abs_coeff': 'ModelLevel',
                   'AbsCoeff': 'ModelLevel',
                   'DDEP': 'Surface', 'Emis': 'Surface',
                   'COLUMN': 'Column', 'D3': 'ModelLevel',
                   'SURF': 'Surface', 'WDEP': 'Surface',
                  'z3d': 'Surface',
                  'Z': 'Surface'}

# Aerocom -> EMEP variable mapping
map_aero_emep = reader.map_aero_emep()

In [14]:
# Loop through all known variables and stores in Aerocom format

for key, var in map_aero_emep.items():
    print('Converting {} / {}\n'.format(key, var))
    # Load data
    try:
        data = reader.read_var(key, ts_type='monthly')
    except KeyError as e:
        print('{} not found in EMEP'.format(key))
        print(repr(e))
        continue
    except ValueError as e:
        print(repr(e))
        continue
    data.change_base_year(year - 110) # Change year to match emission year
    # Infer vertical coordinate from variable naming
    type_emep = (var.split('_')[0])
    vert_code = map_prefix_aero[type_emep]
    data.to_netcdf(out_dir, vert_code=vert_code);
    print('\n')

NameError: name 'map_aero_emep' is not defined